In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import random as rd
from time import sleep
from datetime import datetime
import re

# Initialize the WebDriver for Chrome browser and maximize the browser window.
driver = webdriver.Chrome()
driver.maximize_window()

# URL for the Crunchbase login page.
login_url = 'https://www.crunchbase.com/login'

# Function to handle logging into Crunchbase.
def login_to_crunchbase(driver, email, password):
    # Navigate to the Crunchbase login page.
    driver.get(login_url)

    # Wait for the email input field to be visible and locate it.
    WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, 
                                      "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/authentication-page/page-layout/div/div/div[2]/authentication/mat-card/mat-tab-group/div/mat-tab-body[1]/div/login/form/mat-form-field[1]/div[1]/div/div[2]/input"
                                      )))
    email_input = driver.find_element(By.XPATH, 
                                      "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/authentication-page/page-layout/div/div/div[2]/authentication/mat-card/mat-tab-group/div/mat-tab-body[1]/div/login/form/mat-form-field[1]/div[1]/div/div[2]/input"
                                      )
    # Enter the email.
    email_input.send_keys(email)

    # Wait for the password input field to be visible and locate it.
    WebDriverWait(driver, 4).until(EC.visibility_of_element_located((By.XPATH,
                                                                     "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/authentication-page/page-layout/div/div/div[2]/authentication/mat-card/mat-tab-group/div/mat-tab-body[1]/div/login/form/mat-form-field[2]/div[1]/div/div[2]/input"
                                                                     )))
    password_input = driver.find_element(By.XPATH, 
                                         "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/authentication-page/page-layout/div/div/div[2]/authentication/mat-card/mat-tab-group/div/mat-tab-body[1]/div/login/form/mat-form-field[2]/div[1]/div/div[2]/input"
                                        )
    # Enter the password.
    password_input.send_keys(password)

    # Prompt the user to manually enter credentials, as an alternative to auto login
    print("Please enter your email and password manually.")
    input("Press Enter once you've completed entering your details.")

    # Locate and click the login button using JavaScript.
    login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/authentication-page/page-layout/div/div/div[2]/authentication/mat-card/mat-tab-group/div/mat-tab-body[1]/div/login/form/div[2]/button[1]/span[4]")))
    driver.execute_script("arguments[0].click();", login_button)

    # Wait for the URL to change, indicating a successful login.
    WebDriverWait(driver, 10).until(EC.url_changes("<LOGIN_PAGE_URL>"))
    print("Login successful!")
    return driver

# User credentials for login.
email = 'email'       # Replace with your email
password = 'password' # Replace with your password

# Log in to Crunchbase.
driver = login_to_crunchbase(driver, email, password)

# List to store all the scraped Crunchbase data.
all_crunchbase_data = []

# Path to the Excel file containing Crunchbase company URLs.
crunchbase_url_directory = 'crunchbase_company_urls.xlsx'  
df = pd.read_excel(crunchbase_url_directory)

# Loop through each company URL in the Excel file.
for unique_url in df["Crunchbase"]:
    # Skip blank or invalid entries in the URL column.
    if not isinstance(unique_url, str) or unique_url.strip() == "":
        print("Skipping blank URL entry.")
        continue

    # Navigate to the company URL.
    driver.get(unique_url)
    sleep(rd.randint(4, 14))  # Random delay to mimic human interaction.

    # Scrape various details about the company.
    try:
        Name = driver.find_element(By.XPATH, "//h1").text
    except:
        Name = ""

    try:
        company_location = driver.find_element(By.XPATH, "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/ng-component/entity-v2/page-layout/div/div/mat-tab-nav-panel/div/full-profile/page-centered-layout[1]/div/row-card/div/div[1]/profile-section/section-card/mat-card/div[2]/fields-card/ul/li[1]/label-with-icon").text
    except:
        company_location = ""

    try:
        no_of_employees = driver.find_element(By.XPATH, "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/ng-component/entity-v2/page-layout/div/div/mat-tab-nav-panel/div/full-profile/page-centered-layout[1]/div/row-card/div/div[1]/profile-section/section-card/mat-card/div[2]/fields-card/ul/li[2]/label-with-icon/span/field-formatter/a").text
    except:
        no_of_employees = ""

    try:
        website = driver.find_element(By.XPATH, "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/ng-component/entity-v2/page-layout/div/div/mat-tab-nav-panel/div/full-profile/page-centered-layout[1]/div/row-card/div/div[1]/profile-section/section-card/mat-card/div[2]/fields-card/ul/li[5]/label-with-icon/span/field-formatter/link-formatter/a").get_attribute("href")
    except:
        website = ""

    # Navigate to the financials page of the company.
    financials_url = unique_url + "/company_financials"
    driver.get(financials_url)
    sleep(rd.randint(4, 14))

    # Scrape financial details of the company.
    try:
        total_funding = driver.find_element(By.XPATH, "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/ng-component/entity-v2/page-layout/div/div/mat-tab-nav-panel/div/full-profile/page-centered-layout[1]/div/row-card/div/div[2]/profile-section/section-card/mat-card/div[2]/phrase-list-card[1]/obfuscation/markup-block/field-formatter[2]/a").text
    except:
        total_funding = ""

    try:
        funding_rounds = driver.find_element(By.XPATH, "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/ng-component/entity-v2/page-layout/div/div/mat-tab-nav-panel/div/full-profile/page-centered-layout[1]/div/row-card/div/div[2]/profile-section/section-card/mat-card/div[2]/phrase-list-card[1]/obfuscation/markup-block/field-formatter[3]/a").text
    except:
        funding_rounds = ""

    try:
        funding_seeds = driver.find_element(By.XPATH, "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/ng-component/entity-v2/page-layout/div/div/mat-tab-nav-panel/div/full-profile/page-centered-layout[1]/div/row-card/div/div[2]/profile-section/section-card/mat-card/div[2]/phrase-list-card[1]/obfuscation/markup-block/field-formatter[4]/a").text
    except:
        funding_seeds = ""

    try:
        funding_date = driver.find_element(By.XPATH, "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/ng-component/entity-v2/page-layout/div/div/mat-tab-nav-panel/div/full-profile/page-centered-layout[1]/div/row-card/div/div[2]/profile-section/section-card/mat-card/div[2]/phrase-list-card[1]/obfuscation/markup-block/field-formatter[5]/a").text
    except:
        funding_date = ""

    try:
        last_round_raised = driver.find_element(By.XPATH, "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/ng-component/entity-v2/page-layout/div/div/mat-tab-nav-panel/div/full-profile/page-centered-layout[2]/div/div/div[1]/row-card[1]/profile-section/section-card/mat-card/div[2]/list-card/div/table/tbody/tr[1]/td[4]/field-formatter/span").text
    except:
        last_round_raised = ""

    try:
        last_round_year = driver.find_element(By.XPATH, "/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/ng-component/entity-v2/page-layout/div/div/mat-tab-nav-panel/div/full-profile/page-centered-layout[2]/div/div/div[1]/row-card[1]/profile-section/section-card/mat-card/div[2]/list-card/div/table/tbody/tr[1]/td[1]/field-formatter/span").text
    except:
        last_round_year = ""

    # Create a dictionary to store the scraped data for the current company.
    crunchbase_links = {
        "Crunchbase": unique_url,
        "Name": Name,
        "company_location": company_location,
        "no_of_employees": no_of_employees,
        "Website": website,
        "total_funding": total_funding,
        "funding_rounds": funding_rounds,
        "funding_seeds": funding_seeds,
        "funding_date": funding_date,
        "last_round_raised": last_round_raised,
        "last_round_year": last_round_year,
    }

    # Append the scraped data to the list.
    all_crunchbase_data.append(crunchbase_links)
    print(f"Scrape done: {len(all_crunchbase_data)}")

# Create a DataFrame from the scraped data.
scraped_df = pd.DataFrame(all_crunchbase_data)

# Merge the scraped data with the original data from the Excel file.
final_df = pd.merge(df, scraped_df, on="Crunchbase", how="left")

# Save the merged data to a new Excel file.
final_df.to_excel("crunchbase_scraped_output.xlsx", index=False)

# Close the browser.
driver.quit()